In [6]:
from langchain_ollama.chat_models import ChatOllama
from langchain.tools import tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions
import subprocess

@tool("execute_command")
def execute_command(command):
    """
        This tool takes only one linux commands you want to  
        execute as a string and executes it and returns 
        the output
        Args: 
            command as string 
        Returns: 
            the result of command as string
    """

    try: 
        print(f">> Executing Command \"{command}\"")
        output = subprocess.run(command, shell=True, capture_output=True, text=True).stdout
        return output
    except Exception as e :    
        return f"something went wrong {e}"

In [2]:
def select_tool(tools, prompt): 
    agent = OllamaFunctions(
        model="llama3-groq-tool-use",
        temperature=0,
        format="json",
    ).bind_tools(tools)

    response = agent.invoke(prompt)
    return response.tool_calls

In [3]:
calls = select_tool([execute_command],"print the current working directory")
calls

[{'name': 'execute_command',
  'args': {'command': 'pwd'},
  'id': 'call_73259d3d39b64bcd82f69f0d2440e468',
  'type': 'tool_call'}]

In [5]:
args = calls[0]['args'] 
print(execute_command())

>> Executing Command "pwd"
/home/ha1st/github/Open-Source-ML



In [31]:
from langchain.agents import create_react_agent, AgentExecutor, AgentType
from langchain_ollama.chat_models import ChatOllama
from langchain_community.tools import ShellTool
from langchain import hub

model = ChatOllama(
    model="llama3-groq-tool-use",
    temperature=0,
).bind_tools([execute_command])

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(
    llm=model,
    tools=[execute_command],
    prompt=prompt
)

In [28]:
executor = AgentExecutor(
    agent=agent,
    tools=[execute_command],
    verbose=True, 
    handle_parsing_errors=True
)

In [29]:
executor.invoke({"input": "print the current working directory"})



> Entering new AgentExecutor chain...
Action: execute_command
Action Input: "pwd>> Executing Command "pwd"
/home/ha1st/github/Open-Source-ML
I'm sorry but I do not have enough information to complete your request. Can you provide more details?Invalid Format: Missing 'Action:' after 'Thought:I can help with that. Let me execute the command for you.

Question: print the current working directory
Thought: Action: execute_command
Action Input: "pwd>> Executing Command "pwd"
/home/ha1st/github/Open-Source-ML
Invalid Format: Missing 'Action:' after 'Thought:Invalid Format: Missing 'Action:' after 'Thought:Invalid Format: Missing 'Action:' after 'Thought:Invalid Format: Missing 'Action:' after 'Thought:Invalid Format: Missing 'Action:' after 'Thought:

KeyboardInterrupt: 